In [ ]:
pip install tensorflow-probability

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# **Create training and evaluation datasets**

In [ ]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

# **Compile, train, and evaluate the model**

In [ ]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


# **Create model inputs**

In [ ]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

# **Experiment 1: standard neural network**


In [ ]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wine_quality/white/1.0.0.incomplete7SLC56/wine_quality-train.tfrecord*...:…

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 1s 19ms/step - loss: 39.9586 - root_mean_squared_error: 6.3213 - val_loss: 38.1617 - val_root_mean_squared_error: 6.1775
Epoch 2/100
17/17 [==============================] - 0s 13ms/step - loss: 37.6959 - root_mean_squared_error: 6.1397 - val_loss: 36.6672 - val_root_mean_squared_error: 6.0553
Epoch 3/100
17/17 [==============================] - 0s 12ms/step - loss: 35.7958 - root_mean_squared_error: 5.9830 - val_loss: 35.1770 - val_root_mean_squared_error: 5.9310
Epoch 4/100
17/17 [==============================] - 0s 10ms/step - loss: 33.9886 - root_mean_squared_error: 5.8300 - val_loss: 33.6705 - val_root_mean_squared_error: 5.8026
Epoch 5/100
17/17 [==============================] - 0s 13ms/step - loss: 32.2324 - root_mean_squared_error: 5.6774 - val_loss: 32.1670 - val_root_mean_squared_error: 5.6716
Epoch 6/100
17/17 [==============================] - 0s 13ms/step - loss: 30.5103 - root_mean_squared_

In [ ]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.1 - Actual: 7.0
Predicted: 6.1 - Actual: 7.0
Predicted: 4.9 - Actual: 5.0
Predicted: 6.0 - Actual: 6.0
Predicted: 5.1 - Actual: 4.0
Predicted: 5.8 - Actual: 6.0
Predicted: 5.8 - Actual: 6.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.9 - Actual: 7.0
Predicted: 5.4 - Actual: 5.0


# **BAYESIAN NEURAL NETWORK** 

In [ ]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [ ]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 224ms/step - loss: 38.1226 - root_mean_squared_error: 6.1732 - val_loss: 37.7212 - val_root_mean_squared_error: 6.1408
Epoch 2/500
5/5 [==============================] - 0s 19ms/step - loss: 37.7233 - root_mean_squared_error: 6.1409 - val_loss: 31.3461 - val_root_mean_squared_error: 5.5975
Epoch 3/500
5/5 [==============================] - 0s 17ms/step - loss: 36.0623 - root_mean_squared_error: 6.0040 - val_loss: 38.5598 - val_root_mean_squared_error: 6.2086
Epoch 4/500
5/5 [==============================] - 0s 16ms/step - loss: 41.5241 - root_mean_squared_error: 6.4429 - val_loss: 37.7552 - val_root_mean_squared_error: 6.1433
Epoch 5/500
5/5 [==============================] - 0s 16ms/step - loss: 36.8646 - root_mean_squared_error: 6.0704 - val_loss: 38.7713 - val_root_mean_squared_error: 6.2256
Epoch 6/500
5/5 [==============================] - 0s 16ms/step - loss: 35.9499 - root_mean_squared_error: 5.99

In [ ]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 6.21, min: 5.76, max: 6.43, range: 0.67 - Actual: 7.0
Predictions mean: 6.17, min: 5.75, max: 6.42, range: 0.67 - Actual: 7.0
Predictions mean: 4.8, min: 4.04, max: 5.65, range: 1.61 - Actual: 5.0
Predictions mean: 6.04, min: 5.46, max: 6.37, range: 0.92 - Actual: 6.0
Predictions mean: 5.33, min: 4.58, max: 5.94, range: 1.37 - Actual: 4.0
Predictions mean: 5.68, min: 5.0, max: 6.35, range: 1.36 - Actual: 6.0
Predictions mean: 5.42, min: 4.84, max: 6.02, range: 1.18 - Actual: 6.0
Predictions mean: 6.03, min: 5.5, max: 6.43, range: 0.93 - Actual: 6.0
Predictions mean: 6.01, min: 5.49, max: 6.33, range: 0.84 - Actual: 7.0
Predictions mean: 5.3, min: 4.64, max: 6.04, range: 1.4 - Actual: 5.0


In [ ]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 4s 81ms/step - loss: 37.9190 - root_mean_squared_error: 6.1575 - val_loss: 36.6422 - val_root_mean_squared_error: 6.0530
Epoch 2/500
17/17 [==============================] - 0s 8ms/step - loss: 36.0433 - root_mean_squared_error: 6.0033 - val_loss: 33.3692 - val_root_mean_squared_error: 5.7764
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 36.1784 - root_mean_squared_error: 6.0145 - val_loss: 33.5123 - val_root_mean_squared_error: 5.7886
Epoch 4/500
17/17 [==============================] - 0s 7ms/step - loss: 34.6070 - root_mean_squared_error: 5.8824 - val_loss: 31.6246 - val_root_mean_squared_error: 5.6232
Epoch 5/500
17/17 [==============================] - 0s 12ms/step - loss: 32.4645 - root_mean_squared_error: 5.6974 - val_loss: 30.3082 - val_root_mean_squared_error: 5.5049
Epoch 6/500
17/17 [==============================] - 0s 7ms/step - loss: 31.5274 - root_mean_squared_erro